# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [22]:
! pip install -U langchain==0.2.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/6a/3a/66115e1986e21f7a4a855df2c769c35bea84ca26b31f4e08951a364bb983/langchain-0.2.0-py3-none-any.whl (973 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.8
    Uninstalling langchain-0.2.8:
      Successfully uninstalled langchain-0.2.8


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.6 requires langchain<0.3.0,>=0.2.6, but you have langchain 0.2.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [23]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [24]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位卓越的生物学家。
你对生物学领域的知识了如指掌。 
因为你擅长观察自然界的细微差别，总能从生物样本中发现规律和联系， 你先分析这些独立的生物现象，然后综合这些信息以解答更复杂的生物学问题。

这是一个问题： {input}"""

it_template = """你是一位资深的IT专家。你对信息技术充满热情。 
你的专长在于透彻理解软件和硬件的工作机制，并且能够解析复杂的编程难题。 你通常会先处理基础的技术问题，然后把它们组合起来解决更大规模的IT挑战。

这是一个问题： {input}"""

language_template = """你是一位语言学的大师。你精通多种语言及其语言学背景。 
你的能力源于对语言结构和用法的深刻理解，这使你能够细致分析语言组件， 你首先专注于这些组件的独立意义，然后将它们串联起来深入探讨更广泛的语言学问题。

这是一个问题： {input}"""

In [25]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机学科问题",
        "prompt_template": it_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": math_template,
    },
]

In [26]:

# llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

llm = OpenAI()

In [27]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [28]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [29]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [30]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [31]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机学科问题', '汉语言文学: 适用于回答汉语言文学问题']


In [32]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机学科问题
汉语言文学: 适用于回答汉语言文学问题


In [33]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [34]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机学科问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}


In [35]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [36]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指在一定温度下，完美吸收和辐射所有波长的电磁辐射的物体。它的能量分布与温度有关，符合普朗克定律。黑体辐射是理想情况下的辐射体，可以用来研究热辐射和量子理论。'}


In [38]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要找到大于40的第一个质数。质数是只能被1和自身整除的自然数，因此我们可以从41开始逐个检查是否为质数。\n\n41是质数，但是41加一为42，不能被3整除。因此，我们继续向后寻找。\n\n43是质数，并且43加一为44，能被3整除。因此，大于40的第一个质数是43。\n\n因此，答案为43。'}


In [17]:
router_chain.verbose = True

In [18]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种极端密度和强大引力的天体，它的重力场非常强大，甚至连光也无法逃脱。它的存在是由于恒星在死亡时发生了坍缩，使得它们的质量集中在极小的空间内。黑洞的存在已经被多项观测和理论证据所证实，但仍然有很多未解之谜。'}


In [39]:
print(
    chain.invoke(
        "线粒体和叶绿体的区别和相同的是什么？"
    )
)



> Entering new MultiPromptChain chain...
生物: {'input': '线粒体和叶绿体的区别和相同的是什么？'}
> Finished chain.
{'input': '线粒体和叶绿体的区别和相同的是什么？', 'text': '\n\n线粒体和叶绿体都是细胞中的细胞器，都具有自主复制的能力。它们都是通过细胞的内质网和高尔基体的膜运输到细胞中，都拥有自己的DNA，可以自主合成部分蛋白质。但它们的功能和结构有所不同。\n\n相同点：\n1. 都是细胞中的细胞器，具有自主复制的能力；\n2. 都通过细胞的内质网和高尔基体的膜运输到细胞中；\n3. 都拥有自己的DNA，可以自主合成部分蛋白质。\n\n不同点：\n1. 线粒体是细胞中的能量中心，负责细胞内的呼吸作用，产生能量；而叶绿体则是细胞'}


In [43]:
print(
    chain.invoke(
        "想要学习langchain开发，你有什么学习路线推荐吗？"
    )
)



> Entering new MultiPromptChain chain...
计算机: {'input': '想要学习langchain开发，你有什么学习路线推荐吗？'}
> Finished chain.
{'input': '想要学习langchain开发，你有什么学习路线推荐吗？', 'text': '\n\n作为一位资深的IT专家，我认为学习langchain开发需要掌握以下几个步骤：\n\n1. 了解区块链技术：langchain是基于区块链技术的开发平台，因此首先需要对区块链技术有一定的了解。可以通过阅读相关的书籍、文章或者参加线上的课程来学习。\n\n2. 学习智能合约开发：langchain使用智能合约作为开发语言，因此需要学习智能合约的开发。可以选择学习Solidity语言，它是目前最流行的智能合约开发语言。\n\n3. 掌握开发工具：langchain提供了一套完整的开发工具包，包括IDE、测试工具等。因此需要掌握这些工具的使用方法，以便更高效地开发应用程序。\n\n4.'}


In [42]:
print(
    chain.invoke(
        "道可道，所谓非常道，这个道究竟是什么？"
    )
)



> Entering new MultiPromptChain chain...
汉语言文学: {'input': '道可道，所谓非常道，这个道究竟是什么？'}
> Finished chain.
{'input': '道可道，所谓非常道，这个道究竟是什么？', 'text': '\n\n这是老子《道德经》中的第一句话。道可道，即指可以被言说、表达的道理。所谓非常道，即指这个道理本身无法被言说、表达。因此，道是指万物的本源，无法被言说、描述。它是一种超越言语和概念的存在，只能通过心灵的领悟来理解。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains